In [15]:
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import ShortTermFeatures
import os

def process_directory(directory_path):
    audio_files = [f for f in os.listdir(directory_path) if f.endswith('.mp3') or f.endswith('.wav')]

    all_features = []
    
    for audio_file in audio_files:
        filepath = os.path.join(directory_path, audio_file)
        [Fs, x] = audioBasicIO.read_audio_file(filepath) # nécessaire!
        x = audioBasicIO.stereo_to_mono(x)

        short_term_features = ShortTermFeatures.feature_extraction(x, Fs, 0.050*Fs, 0.025*Fs)
        all_features.append(short_term_features)
        return all_features



In [ ]:
list_features = process_directory("d")

In [ ]:
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import ShortTermFeatures
import matplotlib.pyplot as plt


[Fs, x] = audioBasicIO.read_audio_generic("./d/0.wav")
x = audioBasicIO.stereo_to_mono(x)

F, f_names = ShortTermFeatures.feature_extraction(x, Fs, 0.050*Fs, 0.025*Fs)

plt.subplot(2,1,1); plt.plot(F[0,:]); plt.xlabel('Frame no'); plt.ylabel(f_names[0]) 
plt.subplot(2,1,2); plt.plot(F[1,:]); plt.xlabel('Frame no'); plt.ylabel(f_names[1]); plt.show()